# 🎨 Qwen 图像编辑器 - Gradio版

基于 Qwen-Image-Edit 模型的交互式图像编辑应用

In [ ]:
# 安装必要的依赖
!pip install gradio modelscope diffusers transformers accelerate

In [ ]:
import gradio as gr
import torch
from PIL import Image
import os
from diffusers import QwenImageEditPipeline
from modelscope import snapshot_download
import numpy as np

In [ ]:
# 设置设备和数据类型
if torch.cuda.is_available():
    device = "cuda"
    torch_dtype = torch.bfloat16
    print(f"✅ 使用 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 GPU 内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    device = "cpu"
    torch_dtype = torch.float32
    print("⚠️ 使用 CPU（建议使用GPU以获得更好性能）")

print(f"🔧 设备: {device}, 数据类型: {torch_dtype}")

In [ ]:
# 下载并加载 Qwen-Image-Edit 模型
model_id = "Qwen/Qwen-Image-Edit"
local_dir = './models/Qwen-Image-Edit'

# 检查模型是否已存在
if not os.path.exists(local_dir):
    print(f"📥 开始下载 {model_id} 模型...")
    os.makedirs(os.path.dirname(local_dir), exist_ok=True)
    snapshot_download(model_id, local_dir=local_dir)
    print(f"✅ 模型下载完成: {local_dir}")
else:
    print(f"✅ 模型已存在: {local_dir}")

# 加载管道
print("🔄 正在加载图像编辑管道...")
pipeline = QwenImageEditPipeline.from_pretrained(
    local_dir, 
    torch_dtype=torch_dtype,
    use_safetensors=True,
    device_map="balanced"
)
print(pipeline)

In [ ]:
def edit_image_with_prompt(image, prompt, seed=42, steps=50, cfg_scale=4.0):
    """
    使用提示词编辑图像
    """
    try:
        # 处理输入图像
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        
        if image.mode != "RGB":
            image = image.convert("RGB")
        
        # 编辑参数
        inputs = {
            "image": image,
            "prompt": prompt,
            "generator": torch.manual_seed(seed),
            "true_cfg_scale": cfg_scale,
            "negative_prompt": " ",
            "num_inference_steps": steps,
        }
        
        # 执行编辑
        with torch.inference_mode():
            output = pipeline(**inputs)
            edited_image = output.images[0]
        
        return edited_image
        
    except Exception as e:
        print(f"❌ 编辑图像时出错: {str(e)}")
        return image

In [ ]:
# 创建 Gradio 界面
def create_image_editor():
    with gr.Blocks(title="Qwen 图像编辑器", theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 🎨 Qwen 智能图像编辑器
            
            使用自然语言描述您想要的编辑效果，AI 将为您智能编辑图像！
            
            **使用步骤：** 上传图像 → 输入编辑提示词 → 点击编辑按钮
            """
        )
        
        with gr.Row():
            with gr.Column(scale=1):
                # 输入区域
                input_image = gr.Image(
                    label="📷 上传图像",
                    type="pil",
                    height=350
                )
                
                prompt_input = gr.Textbox(
                    label="✏️ 编辑提示词",
                    placeholder="请详细描述您想要的编辑效果，例如：将背景改为蓝天白云，添加一只可爱的小猫...",
                    lines=3
                )
                
                # 快速示例按钮
                gr.Markdown("**🚀 快速示例：**")
                with gr.Row():
                    btn1 = gr.Button("🎨 动漫风格", size="sm")
                    btn2 = gr.Button("🌅 日落背景", size="sm")
                    btn3 = gr.Button("🐱 添加小猫", size="sm")
                    btn4 = gr.Button("❄️ 冬季雪景", size="sm")
                
                # 参数设置
                with gr.Accordion("🎛️ 高级参数", open=False):
                    seed_input = gr.Slider(0, 1000, value=42, step=1, label="随机种子")
                    steps_input = gr.Slider(10, 100, value=50, step=5, label="推理步数")
                    cfg_scale_input = gr.Slider(1.0, 10.0, value=4.0, step=0.5, label="CFG缩放")
                
                edit_button = gr.Button("🎨 开始编辑", variant="primary", size="lg")
            
            with gr.Column(scale=1):
                # 输出区域
                output_image = gr.Image(label="✨ 编辑结果", height=350)
        
        # 示例提示词说明
        with gr.Accordion("💡 提示词示例", open=False):
            gr.Markdown(
                """
                **风格转换：**
                - "Transform into anime/manga style with vibrant colors"
                - "Convert to oil painting style"
                - "将这张照片转换为水彩画风格"
                
                **场景编辑：**
                - "Change background to beautiful sunset"
                - "Add snow and winter atmosphere"
                - "将白天改为夜晚，添加灯光"
                
                **对象编辑：**
                - "Add a cute cat in the foreground"
                - "Remove the person from image"
                - "在图像中添加一朵美丽的花"
                """
            )
        
        # 编辑处理函数
        def process_edit(image, prompt, seed, steps, cfg_scale, progress=gr.Progress(track_tqdm=True)):
            if image is None:
                return None
            if not prompt.strip():
                return None
            
            try:
                # 执行编辑
                edited_image = edit_image_with_prompt(image, prompt, seed, steps, cfg_scale)
                return edited_image
                
            except Exception as e:
                return None
        
        
        # 绑定事件
        edit_button.click(
            fn=process_edit,
            inputs=[input_image, prompt_input, seed_input, steps_input, cfg_scale_input],
            outputs=[output_image]
        )
        
        # 示例按钮绑定
        examples = [
            "Transform this image into anime/manga style with vibrant colors and soft lighting",
            "Change the background to a beautiful sunset with warm golden and orange colors",
            "Add a cute cat sitting naturally in the foreground of this image",
            "Transform this scene to winter with snow covering everything"
        ]
        
        for btn, example in zip([btn1, btn2, btn3, btn4], examples):
            btn.click(fn=lambda x=example: x, outputs=[prompt_input])
    
    return demo

In [ ]:
print("🚀 启动 Qwen 图像编辑器...")
    
demo = create_image_editor()

# 启动 Gradio 应用
demo.launch(
    share=False,          # 创建公共链接，便于远程访问
    server_name="0.0.0.0",  # 允许外部访问
    server_port=6006,    # 端口号
    show_error=True,     # 显示详细错误信息
    debug=True          # 生产环境建议关闭debug
)